In [15]:
import math
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt 
import sys, os 
import time
import csv
%matplotlib inline

In [16]:
# constants 
EPS0 = 8.85 * 10 ** -12
QE = 1.602 * 10 ** -19 
K = 1.381 * 10 ** -23 
AMU = 1.661 * 10 ** -27
MI = 40 * AMU #mass of an Ar+ ion 

#simulation inputs 
N = 2000 #max particles 
n0 = 1 * 10 ** 12 #density
V_ref = 0 #reference potential 
Te = 1  #electron temperature in eV 
Ti = 0.1 #ion temperature in eV
V_d = -5.1 #dust potential (use values from lit)
v_drift = 7000
tol = 1 * 10 ** -1
iterations = 50


#plasma parameters 
lambd_D = np.sqrt( EPS0 * Te /(n0 * QE) )
vth = np.sqrt((2 * QE * Ti) / MI)
v0 = 0.2 #stream velocity 


#domain parameters 
nx = 10
ny = 10
J = nx * ny 
dx = dy = dh = lambd_D
Lx = nx * dx 
Ly = ny * dy 
dt = (0.1 * dx) / v_drift 
np_in = (ny-1) * 15 

#specific weight 
flux = n0 * v_drift * Ly 
npt = flux * dt #particles created per timestep 
sw = npt / np_in #specific weight 
q_mp = 1 #macroparticle charge 

# particle arrays
p_pos = np.zeros([N,2])
p_velo = np.zeros([N,2])

print(len(p_pos), type(p_pos), sw, q_mp, dh)

2000 <class 'numpy.ndarray'> 409210.708836 1 0.00743259347017


In [17]:
start = time.clock() # Keeping track of computational time 

def charge_depo(dh, NP, J):

    q = np.zeros((nx, ny))
    rho = np.zeros((ny, nx))

    for p in range (1, NP): 
            
            fi = (1 + p_pos[p,0]) / (dh) 
            i = np.floor(fi)
            hx = fi - i  
                
            fj = (1 + p_pos[p,1]) / (dh)
            j = np.floor(fj)
            hy = fj - j 
                        
            q[i,j] = q[i,j] + (1-hx) * (1-hy)
            q[i+1, j] = q[i+1, j] + hx * (1-hy)
            q[i, j+1] = q[i, j+1] + (1-hx) * hy 
            q[i+1, j+1] = q[i+1, j+1] + hx * hy 
            
            rho = (sw + q_mp * q) / (dh * dh)
                
            rho[0,:] = 2 * rho[0,:]
            rho[-1, :] = 2 * rho[-1, :]
            rho[:, 0] = 2 * rho[:, 0]
            rho[:, -1] = 2 * rho[:, -1]
            
            rho = rho + (1 * 10 ** 4)
            #print(rho)
    return q, rho 

print("Clocking in at %s seconds"  % (time.clock() - start))

Clocking in at 0.0007929999999998216 seconds


In [18]:
start = time.clock() # Keeping track of computational time 

#stencil array used in potential solver 

def stencil_fd(J, dx, dh, nx, ny):
    """Uses the finite difference scheme"""
    #E = []
    
    #source = rho
    M = np.zeros((J,J))
    
    for i in range (1, nx-1):
        for j in range(1, ny-1): 
            u = (j-1) * nx + i
            M[u, u] = -4. / (dh * dh) 
            M[u, u-1] = 1. / (dh * dh)
            M[u, u+1] = 1. / (dh * dh)
            M[u, u-nx] = 1. / (dh * dh)
            M[u, u+ (nx-2)] = 1. / (dh * dh)    
    
    for i in range (0, nx):
        u = i+1
        M[u,u] = -1. / dh
        M[u,u+nx] = 1. / dh 
        
    for i in range (0,nx): 
        u = (ny-1) * nx  + i 
        M[u, u-nx] = 1. / dh
        M[u, u] = -1. / dh 
        
    for j in range (0, ny): 
        u = (j-1) * nx + nx 
        M[u, :] = np.zeros([1, J])
        M[u, u-1] = 1. / dh 
        M[u, u] = -1. / dh 
        
    for j in range(0, ny): 
        u = (j-1) * nx + 1 
        M[u, :] = np.zeros([1, J])
        M[u, u] = 1.     
        
    return M
print("Clocking in at %s seconds"  % (time.clock() - start))

Clocking in at 0.0004949999999999122 seconds


In [19]:
start = time.clock() # Keeping track of computational time 

def solver_2d(rho, tol, Ti, n0, V_ref, QE, iterations): 
    'Uses Gauss Sidel method'
        
    M = stencil_fd(J, dx, dh, nx, ny)
    V = np.ones((ny,nx)) * V_ref
    
    source = rho
    NX = NY = np.size(source,1)
    NN = np.size(source)
    
    b0 = np.reshape(source, (NN,1))
    x = np.reshape(V, (np.size(V), 1))
    
    #print(np.size(x), np.size(b0))
    
    b = b0 - n0 * np.exp((x-V_ref)/ Ti)
    b = b * QE / EPS0     
    
    for it in range (0, iterations):
        
        b[0:NX] = 0 
        b[NN-NX+1 : NN] = 0 
        b[NX: NX : NN] = 0 
        b[1:NX:NN] = V_ref
        
        if it % 10 == 0:
            R = np.linalg.norm(b-M*x)
            
            if R <= tol:
                print('Solver converged!')
            
    if R > tol: 
        print('Solver failed to converge. Try tweaking your inputs.')
    
    V = np.reshape(x, (NX,NY))
    
    return V
print("Clocking in at %s seconds"  % (time.clock() - start))

Clocking in at 0.00042100000000022675 seconds


In [21]:
NP = 1

for it in range (1, iterations):
    q, rho = charge_depo(dh, NP, J)
    #print(rho, np.size(rho))    
    V = solver_2d(rho, tol, Ti, n0, V_ref, QE, iterations)    

Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your inputs.
Solver failed to converge. Try tweaking your i